In [36]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import pandas as pd
from scipy import stats
from pydataset import data
import numpy as np
import env
import matplotlib.pyplot as plt
import os
import prepare
import acquire
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier

In [37]:
titan_df = acquire.get_titanic_data()
titan_df = titan_df.drop(titan_df.columns[0], axis = 1)
titan_df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [38]:
titan_df = prepare.clean_titanic(titan_df)

In [39]:
titan_df = titan_df.drop(columns = ['sex','embark_town'])

In [40]:
train, val, test = prepare.train_val_test(titan_df, 'survived') #splits up the data
train.head()

,survived,pclass,sibsp,parch,fare,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
748,0,1,1,0,53.1000,0,1,0,1
45,0,3,0,0,8.0500,1,1,0,1
28,1,3,0,0,7.8792,1,0,1,0
633,0,1,0,0,0.0000,1,1,0,1
403,0,3,1,0,15.8500,0,1,0,1


In [41]:
x_train = train.drop(columns = ['survived'])
y_train = train['survived']
x_val = val.drop(columns = ['survived'])
y_val = val['survived']
x_test = test.drop(columns = ['survived'])
y_test = test['survived']

Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

In [42]:
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [43]:
knn.score(x_train,y_train)

0.8314606741573034

In [44]:
knn.score(x_val, y_val)

0.7611940298507462

Evaluate your results using the model score, confusion matrix, and classification report.

In [45]:
train_preds = knn.predict(x_train)
print(classification_report(y_train, train_preds))

              precision    recall  f1-score   support

           0       0.85      0.88      0.87       384
           1       0.80      0.75      0.77       239

    accuracy                           0.83       623
   macro avg       0.82      0.82      0.82       623
weighted avg       0.83      0.83      0.83       623



In [46]:
cm = confusion_matrix(y_train, train_preds)

cm



array([[339,  45],
       [ 60, 179]])

Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

Run through steps 1-3 setting k to 10

In [47]:
num_neighbors = []
train_acc = []
val_acc = []

for i in range (1,11):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(x_train, y_train)
    num_neighbors.append(i)
    train_acc.append(knn.score(x_train, y_train))
    val_acc.append(knn.score(x_val, y_val))
    
pd.DataFrame({'neighbors': num_neighbors,
             'train_acc': train_acc,
             'val_acc': val_acc})

,neighbors,train_acc,val_acc
0,1,0.902087,0.716418
1,2,0.831461,0.731343
2,3,0.831461,0.746269
3,4,0.813804,0.746269
4,5,0.831461,0.761194
5,6,0.815409,0.746269
6,7,0.818620,0.738806
7,8,0.800963,0.753731
8,9,0.791332,0.716418
9,10,0.778491,0.716418


Run through steps 1-3 setting k to 20

In [48]:
num_neighbors = []
train_acc = []
val_acc = []

for i in range (1,21):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(x_train, y_train)
    num_neighbors.append(i)
    train_acc.append(knn.score(x_train, y_train))
    val_acc.append(knn.score(x_val, y_val))
    
df_knn = pd.DataFrame({'neighbors': num_neighbors,
             'train_acc': train_acc,
             'val_acc': val_acc})

df_knn['difference'] = (df_knn['train_acc'] - df_knn['val_acc'])    
df_knn

,neighbors,train_acc,val_acc,difference
0,1,0.902087,0.716418,0.185669
1,2,0.831461,0.731343,0.100117
2,3,0.831461,0.746269,0.085192
3,4,0.813804,0.746269,0.067536
4,5,0.831461,0.761194,0.070267
5,6,0.815409,0.746269,0.069141
6,7,0.818620,0.738806,0.079814
7,8,0.800963,0.753731,0.047232
8,9,0.791332,0.716418,0.074914
9,10,0.778491,0.716418,0.062073


What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?


Which model performs best on our out-of-sample data from validate?

 the model seems to peform the best with k = 16.  after that the model accuracy goes down